In [2]:
import pandas as pd
import os

### Big League Data

In [8]:
# Reading into dict
cwd = os.getcwd()
data_path = os.path.abspath(os.path.join(cwd, "..", "data", "raw"))
    
data_dict = {}
for file in os.listdir(data_path):
    filename = file.split(".")[0]
    filepath = os.path.join(data_path, file)
    
    data_dict[filename] = pd.read_csv(filepath)

In [15]:
# Getting club and league pairing list
league_id_dict = {}

for league, df in data_dict.items():
    clubs = df.club_name.unique().tolist()

    for club in clubs:
        league_id_dict[club] = league

In [51]:
# Mapping club transferred to, to league
for league, df in data_dict.items():
    df.rename(columns={'league_name':'league_from_name'}, inplace=True)

    df["league_to_name"] = df["club_involved_name"].map(league_id_dict)

    # Create internal transfers df, containing only transfers between these leagues
    export_df = df.loc[df["league_to_name"].str.len() > 0]

    export_path = os.path.join(data_path)

Issue - `club_involved_name` does not always match `club_involved_name`, and even so many clubs are outside of Europe anyway - add lower-level European clubs

In [77]:
num_pl_transfers = len(data_dict["premier-league"])

df = data_dict["premier-league"]
df_league = df.loc[df["league_to_name"].str.len() > 0]
num_pl_bigleague_transfers = len(df_league)

# % of premier league transfers going to top leagues
num_pl_bigleague_transfers / num_pl_transfers * 100

15.409822112520214

In [107]:
# PL transfers made between eu clubs as a % of total PL fees 
df_league.fee_cleaned.sum() / data_dict["premier-league"].fee_cleaned.sum() * 100

23.167039410036978

In [106]:
# The required calculation for flows in/out
data_dict["premier-league"].loc[data_dict["premier-league"]["league_to_name"].str.len() > 0].groupby(by=["transfer_movement", "league_to_name"]).sum()

C:\Users\timhy\AppData\Local\Temp\ipykernel_3752\3173271197.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_dict["premier-league"].loc[data_dict["premier-league"]["league_to_name"].str.len() > 0].groupby(by=["transfer_movement", "league_to_name"]).sum()


age  fee_cleaned     year
transfer_movement league_to_name                                 
in                1-bundesliga       4099.0      893.500   328148
                  championship       5709.0       99.561   515769
                  eredivisie         2635.0      350.800   221177
                  liga-nos           2131.0      762.450   175042
                  ligue-1            3988.0     1108.830   341785
                  premier-league    15232.0     1012.824  1240291
                  premier-liga        732.0      164.130    56327
                  primera-division   5306.0     1128.670   426534
                  serie-a            3791.0      771.015   291857
out               1-bundesliga       3130.0      395.300   251706
                  championship       7507.0       22.494   666313
                  eredivisie         1880.0       14.050   158976
                  liga-nos           1264.0       57.210   102653
                  ligue-1            2653.0       87.640   211150
                  premier-league    19188.0      926.028  1545239
                  premier-liga        601.0       56.000    46310
                  primera-division   5251.0      800.215   420537
                  serie-a            3659.0      330.050   276001

### All club data

In [86]:
team_names = pd.read_csv(os.path.join(data_path, "jokecamp_teams.csv"), encoding="utf-8")

In [87]:
team_names

,Name,LeagueCode
0,Shanghai SIPG FC,CHN-D1
1,&Ntilde;ublense,CHI-AP
2,1461 Trabzon,TUR-1LG
3,1461 Trabzon,TUR-CUP
4,1896 Rain Am Lech,GER-RLBV
...,...,...
4717,Örebro SK,SWE-SPR
4718,Örgryte IS,SWE-FA
4719,Örgryte IS,SWE-SPR
4720,Östers IF,SWE-ASV
